In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib

import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [3]:
def calculate_bollinger_bands(data, window=20, num_std_dev=3):
    # Calculate rolling mean and standard deviation
    data['Rolling Mean'] = data['close'].rolling(window=window).mean()
    data['Upper Band'] = data['Rolling Mean'] + (data['close'].rolling(window=window).std() * num_std_dev)
    data['Lower Band'] = data['Rolling Mean'] - (data['close'].rolling(window=window).std() * num_std_dev)
    data['flag'] = np.where(data['close'] > data['Upper Band'], 1.0, 0.0)
    data['flag'] = np.where(data['close'] < data['Lower Band'], -1.0, data['flag'])
    return data

In [4]:
df = pd.read_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\dataset\train\btcusdt_1h_train.csv")


In [5]:
df = calculate_bollinger_bands(df)

In [6]:
def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300', 'tick_10'], aggfunc = ['mean', 'median', 'count'], buckets = 10):
# def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60'], aggfunc = ['mean', 'median', 'count']):
    for alpha in alphas:
        msg = df.pivot_table(index = pd.qcut(df[alpha],buckets,duplicates='drop'), values=rets, aggfunc=aggfunc)
        print(msg)

def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300']):
# def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10']):
# def print_corrs(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300']):
# def print_corrs(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60']):
    msg = "                                      "
    for ret in rets:
        msg += f"{ret:>8s}"
    print(msg)
    for alpha in alphas:
        msg = f"{alpha:30s} corr -> "
        for ret in rets:
            msg += f"{df[alpha].corr(df[ret])*100:7.2f} "
        print(msg)
    print()


In [7]:
df['change_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4

df['ret_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4
df['ret_1'] = df['close'].rolling(window=1).mean().pct_change() * 1e4
df['ret_10'] = df['close'].rolling(window=10).mean().pct_change() * 1e4
df['ret_30'] = df['close'].rolling(window=30).mean().pct_change() * 1e4

df['ret_5'] = df['ret_5'].shift(-5)
df['ret_1'] = df['ret_1'].shift(-1)
df['ret_10'] = df['ret_10'].shift(-10)
df['ret_30'] = df['ret_30'].shift(-30)

df.head(10)
df = df.dropna()
df = df.reset_index(drop=True)

In [8]:
# Remove the rows where the value of flag is 0
df2 = df[df['flag'] != 0.0]

In [9]:
df2.head(6)


,Unnamed: 0,datetime,open,high,low,close,volume,Rolling Mean,Upper Band,Lower Band,flag,change_5,ret_5,ret_1,ret_10,ret_30
23,42,2018-01-02 23:30:00,13789.98,14750.00,13755.43,14580.00,1573.603550,13549.2795,14492.915452,12605.643548,1.0,104.246383,12.930265,104.766804,29.776710,3.906983
157,176,2018-01-08 14:30:00,15471.00,15520.00,14619.00,14650.00,1812.077172,15924.3180,17150.027999,14698.608001,-1.0,-126.227666,-154.279468,364.225256,37.074061,0.111354
162,181,2018-01-08 19:30:00,14659.88,14693.01,13155.00,13489.95,2512.360318,15586.9430,17657.895580,13515.990420,-1.0,-154.279468,236.274129,600.469238,119.698083,28.472769
296,315,2018-01-14 09:30:00,14165.00,14200.00,13537.00,13644.42,1102.573529,14207.4500,14702.905609,13711.994391,-1.0,-79.544946,-34.114650,87.559603,-25.405542,0.138767
328,347,2018-01-15 17:30:00,13645.00,14200.27,13520.17,14195.00,1089.092586,13478.9770,14181.069023,12776.884977,1.0,124.328386,-52.565739,-77.153927,-32.917118,-83.027030
440,459,2018-01-20 09:30:00,11749.91,12384.00,11545.00,12340.00,2016.484805,11482.3325,12304.932036,10659.732964,1.0,148.880530,34.028554,333.071313,20.730805,-15.524207


In [10]:
print_buckets(df2, ['flag'], rets = ['ret_1', 'ret_10', 'ret_5', 'ret_30'], aggfunc = ['mean', 'median', 'count'], buckets = 4000)

                      mean                                   median            \
                     ret_1    ret_10    ret_30     ret_5      ret_1    ret_10   
flag                                                                            
(-1.001, -0.979]  5.925397 -1.314206  0.232401 -2.300789   8.244275 -0.327050   
(0.805, 1.0]      0.341563  4.836129  1.942376  5.039101 -12.086105  2.272503   
(-0.979, -0.756]       NaN       NaN       NaN       NaN        NaN       NaN   
(-0.756, -0.533]       NaN       NaN       NaN       NaN        NaN       NaN   
(-0.533, -0.31]        NaN       NaN       NaN       NaN        NaN       NaN   
(-0.31, -0.087]        NaN       NaN       NaN       NaN        NaN       NaN   
(-0.087, 0.136]        NaN       NaN       NaN       NaN        NaN       NaN   
(0.136, 0.359]         NaN       NaN       NaN       NaN        NaN       NaN   
(0.359, 0.582]         NaN       NaN       NaN       NaN        NaN       NaN   
(0.582, 0.805]         NaN  

In [11]:
# in the row you find flag set the value of row after 1 to -1
# in the row you find flag set the value of row after 1 to -1

df['sig'] = np.nan


In [12]:
for i in range(len(df)):
    if i+9 < len(df):
        if df['flag'].iloc[i] == 1.0:
            df['sig'].iloc[i] = 1.0
            df['sig'].iloc[i+4] = -1.0
        if df['flag'].iloc[i] == -1.0:
            df['sig'].iloc[i+1] = -1.0
            df['sig'].iloc[i+4] = 1.0

In [13]:
compare = 0
df['signal'] = np.nan

for i in range(len(df)):
    if df['sig'].iloc[i] == 1.0 and compare == 0:
        compare = 1
        df['signal'].iloc[i] = 1
    
    elif df['sig'].iloc[i] == -1.0 and compare == 0:
        compare = -1
        df['signal'].iloc[i] = -1

    elif df['sig'].iloc[i] == 1.0 and compare == 1:
        compare = 1
        df['signal'].iloc[i] = 0

    elif df['sig'].iloc[i] == -1.0 and compare == -1:
        compare = -1
        df['signal'].iloc[i] = 0

    elif df['sig'].iloc[i] == 1.0 and compare == -1:
        compare = 0
        df['signal'].iloc[i] = 1

    elif df['sig'].iloc[i] == -1.0 and compare == 1:
        compare = 0
        df['signal'].iloc[i] = -1

df['signal'] = df['signal'].fillna(0)

In [14]:
df['signal'].value_counts()

signal
 0.0    28989
 1.0      385
-1.0      385
Name: count, dtype: int64

In [15]:
def apply_trading_strategy(df, flag_column, log_column, stop_loss_thresh=-0.07):
    exit_loss = 0
    disable_trading = 0
    compare = 0
    df[log_column] = np.nan

    for i in range(len(df)):
        if df[flag_column].iloc[i] == 1 and compare == 0:
            # No open trade, encounter buy signal
            exit_loss = 0
            buy_price = df["open"].iloc[i]
            compare = 1
            df[log_column].iloc[i] = 1

        elif (df[flag_column].iloc[i] != -1 and compare == 1):
            # Current buy trade, encounter buy signal or no signal - update stop loss
            df[log_column].iloc[i] = 0

            # Calculate pnl, if we exit now
            sell_price = df["open"].iloc[i]
            exit_loss = (sell_price - buy_price) / buy_price

            # Exit trade, if the loss is higher than stop loss
            if exit_loss < stop_loss_thresh and disable_trading == 0:
                df[log_column].iloc[i] = -1
                disable_trading = 1

        elif df[flag_column].iloc[i] == -1 and compare == 1:
            # Current buy trade, encounter sell signal

            # Exit trade
            df[log_column].iloc[i] = -1
            compare = 0
            exit_loss = 0

            # If trade was already exited before, check for disable trading flag here, do nothing here
            if disable_trading == 1:
                disable_trading = 0
                df[log_column].iloc[i] = 0

        # SHORT TRADES
        elif df[flag_column].iloc[i] == -1 and compare == 0:
            # No open trade, encounter sell signal
            exit_loss = 0
            sell_price = df["open"].iloc[i]
            compare = -1
            df[log_column].iloc[i] = -1

        elif (df[flag_column].iloc[i] != 1 and compare == -1):
            # Current sell trade, encounter sell signal or no signal - do nothing, update stop loss
            df[log_column].iloc[i] = 0

            # Calculate pnl, if we exit now
            buy_price = df["open"].iloc[i]
            exit_loss = (sell_price - buy_price) / sell_price

            # Exit trade, if the loss is higher than stop loss
            if exit_loss < stop_loss_thresh and disable_trading == 0:
                df[log_column].iloc[i] = 1
                disable_trading = 1

        elif df[flag_column].iloc[i] == 1 and compare == -1:
            # Exit trade
            df[log_column].iloc[i] = 1
            compare = 0
            exit_loss = 0

            # If trade was already exited before, check for disable trading flag here, do nothing here
            if disable_trading == 1:
                disable_trading = 0
                df[log_column].iloc[i] = 0

        elif df[flag_column].iloc[i] == 0 and compare == 0:
            df[log_column].iloc[i] == 0

    # Fill in any missing values in the new column with 0.
    df[log_column].fillna(0, inplace=True)

    # Close out positions (if needed)
    df[log_column].iloc[-1] = -np.sum(df[log_column])

    # df_filter = df[['datetime', 'open', 'high', 'low', 'close', 'volume', log_column]]
    # df_filter.columns = ["datetime", 'open', 'high', 'low', 'close', 'volume', 'signal']
    # df_filter = df_filter.reset_index(drop=True)
    
    return df

# Example usage:
# Load your DataFrame 'df' with the required columns
# df = ...

# Apply the trading strategy function
result_df = apply_trading_strategy(df, flag_column='signal', log_column='signal1')
print(sum(abs(result_df["signal"])))
result_df['signal1'].value_counts()
# Save the result to a CSV file
# result_df.to_csv('output_file.csv', index=False)ig
df['signal'] = result_df['signal']

770.0


In [16]:
# result_df['signal'] = result_df['signal'] * -1

In [17]:
df.to_csv(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\src\logs\bollinger.csv")